## Brochure Website Scraper
A simple Python class to scrape text content from brochure-style websites using requests and BeautifulSoup.

In [65]:
from openai import OpenAI
from website_scraper import Website
from IPython.display import Markdown, display, update_display
import json

In [27]:
from dotenv import load_dotenv
import os

# Load API key from the environment variable `GEMINI_API_KEY`.
load_dotenv(override=True)  
api_key = os.getenv('GEMINI_API_KEY')

In [28]:
gemini = OpenAI(
    api_key=api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

In [30]:
website = Website("https://www.tcs.com/")
website.get_links()

['https://www.tcs.com',
 'https://www.tcs.com/contact-us/whats-on-your-mind',
 'https://www.tata.com/',
 'https://www.tcs.com/what-we-do/perpetually-adaptive-enterprise',
 'https://www.tcs.com/what-we-do.html#industries',
 'https://www.tcs.com/what-we-do.html#services',
 'https://www.tcs.com/what-we-do.html#products',
 'https://www.tcs.com/what-we-do/research',
 'https://www.tcs.com/who-we-are/alliances-partnerships',
 'https://www.tcs.com/what-we-do/industries/banking',
 'https://www.tcs.com/what-we-do/industries/capital-markets',
 'https://www.tcs.com/what-we-do/industries/consumer-packaged-goods-and-distribution',
 'https://www.tcs.com/what-we-do/industries/communications-media-information-services',
 'https://www.tcs.com/what-we-do/industries/education',
 'https://www.tcs.com/what-we-do/industries/energy-resources-utilities',
 'https://www.tcs.com/what-we-do/industries/healthcare',
 'https://www.tcs.com/what-we-do/industries/high-tech',
 'https://www.tcs.com/what-we-do/industries/i

In [31]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [32]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are the top 10 relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    website = Website(url)
    links = website.get_links()
    user_prompt += "\n".join(links)
    return user_prompt

In [34]:
print(get_links_user_prompt("https://www.tcs.com/"))


Here is the list of links on the website https://www.tcs.com/ -
Please decide which of these are the top 10 relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://www.tcs.com
https://www.tcs.com/contact-us/whats-on-your-mind
https://www.tata.com/
https://www.tcs.com/what-we-do/perpetually-adaptive-enterprise
https://www.tcs.com/what-we-do.html#industries
https://www.tcs.com/what-we-do.html#services
https://www.tcs.com/what-we-do.html#products
https://www.tcs.com/what-we-do/research
https://www.tcs.com/who-we-are/alliances-partnerships
https://www.tcs.com/what-we-do/industries/banking
https://www.tcs.com/what-we-do/industries/capital-markets
https://www.tcs.com/what-we-do/industries/consumer-packaged-goods-and-distribution
https://www.tcs.com/what-we-do/industries/communications-media-information-services
https://www.tcs.com/what-we-do/in

In [35]:
def select_relevant_links(url):
    response = gemini.chat.completions.create(
        model="gemini-2.5-flash",
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)},
        ],
        response_format={"type": "json_object"}
    )
    result =  response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [41]:
print(select_relevant_links("https://www.tcs.com/"))

{'links': [{'type': 'homepage', 'url': 'https://www.tcs.com'}, {'type': 'about page', 'url': 'https://www.tcs.com/who-we-are'}, {'type': 'core business offerings', 'url': 'https://www.tcs.com/what-we-do/perpetually-adaptive-enterprise'}, {'type': 'industries overview', 'url': 'https://www.tcs.com/what-we-do#industries'}, {'type': 'services overview', 'url': 'https://www.tcs.com/what-we-do#services'}, {'type': 'products and platforms overview', 'url': 'https://www.tcs.com/what-we-do#products'}, {'type': 'careers page', 'url': 'https://www.tcs.com/careers'}, {'type': 'newsroom', 'url': 'https://www.tcs.com/who-we-are/newsroom'}, {'type': 'corporate social responsibility', 'url': 'https://www.tcs.com/who-we-are/corporate-social-responsibility'}, {'type': 'contact page', 'url': 'https://www.tcs.com/contact-us/whats-on-your-mind'}]}


### Extract text content from all links on the website

```python

In [42]:
def fetch_link_contents(url):
    website_contents = Website(url).text
    relevent_links = select_relevant_links(url)

    result = f"==> Landing Page:\n{website_contents}\n\n==> Relevant Links:\n"

    for link in relevent_links['links']:
        link_url = link['url']
        link_contents = Website(link_url).text
        result += f"### Link: {link['type']}\n\n{link_contents}\n\n"
    
    return result

In [43]:
print(fetch_link_contents("https://www.tcs.com/"))

==> Landing Page:
Skip to main content
Skip to footer
Click here or press enter to go to homepage
What we do
Who we are
Insights
Careers
Newsroom
Investors
Search Button
Contact us
tata.com logo
Open in New Tab
Perpetually Adaptive Enterprise
At TCS, we don’t just help businesses transform. We help them become perpetually adaptive enterprises, built to evolve continuously and confidently in a world of constant change.
Adaptability starts here
Industries
Services
Products and Platforms
Research & Innovation
Alliances
Banking
Capital Markets
Consumer Packaged Goods and Distribution
Communications, Media, and Information Services
Education
Energy, Resources, and Utilities
Healthcare
High Tech
Insurance
Life Sciences
Manufacturing
Public Services
Retail
Travel and Logistics
Artificial Intelligence and Data & Analytics
Cloud
Cognitive Business Operations
Consulting
Cybersecurity
Enterprise Solutions
IoT and Digital Engineering
Network Solutions and Services
Sustainability Services
TCS Inter

In [ ]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_link_contents(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("TCS", "https://www.tcs.com/")

'\nYou are looking at a company called: Apple\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n==> Landing Page:\nApple\nApple\nStore\nMac\niPad\niPhone\nWatch\nVision\nAirPods\nTV & Home\nEntertainment\nAccessories\nSupport\n0\n+\nApple Fitness Plus\nRing in your resolutions.\nGet up to 3 months on us.\n1\nLearn more\nLearn more\nTry it free\nTry it free\nScan code with iPhone or iPad.\nUse the Camera app to activate your free trial.\n1\nApple Watch Series 11\nTurn resolutions into\xa0routines. Quit quitting your fitness\xa0goals.\nLearn more\nBuy\niPhone\nSay hello to the latest generation of iPhone.\nLearn more\nShop iPhone\nAirPods Pro 3\nThe world’s best in-ear Active Noise Cancellation.\nLearn more\nBuy\niPad Air\nNow supercharged by the M3 chip.\nLearn more\nBuy\nMacBook\xa0Pro 14”\nSupercharged by M5.\nLearn more\nBuy\niPad\nNow with the speed of the A16 chip

In [47]:
def create_brochure(company_name, url):
    response = gemini.chat.completions.create(
        model="gemini-2.5-flash",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ]
    )  
    result = response.choices[0].message.content
    display(Markdown(result))

In [48]:
create_brochure("TCS", "https://www.tcs.com/")

# TCS: Your Business, Evolved. (Because who needs a static business in a TikTok world?)

Ever feel like your business is trying to navigate a digital hurricane in a rowboat? Fear not, fellow captains of industry! At TCS, we're not just offering you a bigger boat; we're building you a perpetually adaptive submarine that can not only weather the storm but also enjoy a nice cocktail underwater. We call it a "Perpetually Adaptive Enterprise" – because "always changing and never stressed" was too long for the marketing team.

## For Our Esteemed Customers: The Adaptability Advantage

Think of us as your corporate chameleons, but with a PhD in future-proofing. Whether you're in Banking (where change is as constant as interest rates), Healthcare (where innovation is vital), or even, dare we say, Public Services (yes, even governments can adapt!), we've got you covered. From making your AI actually *smart* to securing your data better than a dragon guarding its hoard, and ensuring your cloud doesn't just store data but *inspires* it – we do it all. We don't just transform your business; we teach it to dance in the digital rain, with services in everything from AI & Analytics to IoT & Digital Engineering, and even Sustainability (because saving the planet is *always* good for business).

## For Our Savvy Investors: The Perpetual Growth Machine

If you're looking for a company that thrives on chaos (the good kind, obviously), then TCS is your golden ticket! We're building businesses that laugh in the face of disruption and evolve faster than viral trends. With a client roster spanning nearly every industry under the sun – from the very serious Capital Markets to the surprisingly dynamic Travel & Logistics – our adaptability isn't just a buzzword; it's our business model. Invest in perpetual motion, because standing still is so last century.

## For Aspiring Innovators & Future Legends: Join the Evolution!

Do you look at "constant change" and see "endless opportunity"? Do you dream in algorithms and wake up thinking about scalable solutions? Then TCS might just be your next great adventure! While we can't promise you a perpetually adaptive desk chair, we can promise a perpetually adaptive environment where your ideas will help shape the businesses of tomorrow.

**Our Culture?** It's built for explorers, problem-solvers, and anyone who enjoys the thrill of continuous evolution. We believe that confidently adapting is the secret to success, and we foster a workplace where learning and innovation are as constant as the coffee supply. If you're ready to embrace continuous evolution, challenge the status quo, and occasionally marvel at how quickly technology changes, then your journey to becoming a perpetually adaptive professional starts here. We're looking for minds that are as flexible as our solutions and as confident as our enterprises.

**The TCS Promise:**
We don't just adapt; we thrive. We don't just transform; we perpetually evolve.
Ready to stop merely existing and start perpetually adapting?

Visit us at tata.com (we help them adapt too!).

## Minor improvements
stream response to show typewriter effect when generating brochure, and switch to gemini-2.5-flash model for better output.

In [71]:
def create_brochure(company_name, url):
    with gemini.chat.completions.stream(
        model="gemini-2.5-flash",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ]
    ) as stream_response:
        result = ""
        display_handle = display(Markdown(""), display_id=True)

        for chunk in stream_response:
            if chunk.type == "content.delta":
                result += chunk.delta
                update_display(Markdown(result), display_id=display_handle.display_id)  # Update display with typewriter effect

In [72]:
create_brochure("TCS", "https://www.tcs.com/")

Welcome to TCS: Where Your Business Learns to Do the Robot – But, Like, Metaphorically and Profitably!

Tired of your business feeling like a flip phone in a smartphone world? Do you dread change more than a cat dreads bath time? Fear not! At **TCS**, we don't just help businesses *transform* (that's so last decade). We help them become **Perpetually Adaptive Enterprises**. Think of it as giving your company a superpower that lets it bend, flex, and occasionally do a graceful little jig in the face of constant change. Because frankly, if you're not evolving, you're just... well, evolving backwards!

### For Our Prospective Customers: Who Needs Perpetual Adaptability? (Hint: You Do!)

Are you in Banking, Energy, Healthcare, Manufacturing, Retail, or... well, pretty much *any* industry under the sun that needs to stay afloat and fabulous? Good news! We're probably already working wonders for your peers (or getting ready to). From AI and Cloud wizardry to Cybersecurity and IoT magic, we're the folks who make sure your business doesn't just survive the future; it owns it, probably with a monocle and a top hat. We make businesses so adaptable, they could probably teach a chameleon a thing or two about blending in (or standing out, depending on the market).

### For Savvy Investors: Put Your Dough Where the Perpetual Motion Is!

Let's face it, in a world of constant flux, stability is the new unicorn. But *adaptability*? That's the golden goose laying perpetually valuable eggs! Investing in TCS means investing in a future where businesses don't just ride the waves of change; they *surf* them, often with a cheeky grin. We're building enterprises that are ready for anything, which means less risk, more resilience, and a rather pleasant upward trend for your portfolio. We're not just predicting the future; we're actively making sure businesses are ready to party in it!

### For Ambitious Recruits: Join the Perpetual Party!

If "constant change" sounds less like a headache and more like your personal jam, and "evolving continuously" is already tattooed on your soul (or at least your LinkedIn profile), then welcome home! At TCS, you'll be at the forefront of innovation, helping craft the very fabric of tomorrow's adaptive enterprises. Whether your brain thrives on Artificial Intelligence, Cloud architecture, Digital Engineering, or making cognitive business operations... well, more cognitive, your perpetually exciting career starts here. We're all about forward momentum, collaboration, and probably a decent amount of coffee. Come build the future with us – it's much more fun than just reacting to it!

So, whether you're looking to turn your business into an agile, unshakeable force, seeking a career where "stagnation" is a forbidden word, or investing in a company that literally makes resilience its business, TCS is your destination. Adaptability starts here. And frankly, it never stops!

In [51]:
!ollama serve

^C


In [68]:
# check if ollama server is running
import requests
requests.get("http://localhost:11434").content

b'Ollama is running'

In [69]:
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [70]:
def stream_response():
    with ollama.chat.completions.stream(
        model="llama3.2",
        messages=[
            {"role": "system", "content": "you are a helpful assistant."},
            {"role": "user", "content": "Hi there! tell me a fun fact",}
        ]
    ) as stream_response:
        result = ""
        display_handle = display(Markdown(""), display_id=True)

        for chunk in stream_response:
            if chunk.type == "content.delta":
                result += chunk.delta
                update_display(Markdown(result), display_id=display_handle.display_id)  # Update display with typewriter effect

stream_response()

Here's one:

Did you know that there is a species of jellyfish that is immortal?! The Turritopsis dohrnii, also known as the "immortal jellyfish," can transform its body into a younger state through a process called transdifferentiation. This means it can essentially revert back to its polyp stage and start its life cycle all over again! Isn't that mind-blowing?

Would you like to hear more fun facts?